ใน ep นี้เราจะมาเรียนรู้วิธีการใช้ประโยชน์จากแผนที่มากขึ้น หนึ่งในงานที่เราจะใช้ัแผนที่ คือ Geocode ข้อมูลชื่อที่อยู่สถานที่ ให้เป็นตำแหน่งพิกัดของสถานที่จริง ที่อยู่บนพื้นผิวโลก และการ Spatial Join คือ การ Match พิกัดเข้าด้วยกันระหว่าง 2 Shape

# 0. Install

เราจะต้อง Install kaggle เพื่อ Download Dataset, geopandas เพื่อใช้ในการวิเคราะห์ข้อมูล geospatial, folium เพื่อแสดงแผ่นที่, rtree ให้สำหรับ Spatial Join (ถ้ายังไม่ได้ Install ให้ uncomment)

In [0]:
# !apt install gdal-bin python-gdal python3-gdal 
# # Install rtree - Geopandas requirment
# !apt install python3-rtree 
# # Install Geopandas
# !pip install git+git://github.com/geopandas/geopandas.git
# # Install descartes - Geopandas requirment
# !pip install descartes 

In [0]:
# ! pip install git+https://github.com/python-visualization/folium
# ! pip install kaggle --upgrade

# 1. Import Library

Import Library เพื่อใช้ในการวิเคราะห์ Spatial Analysis และ พล็อตแผนที่แบบ Interactive

In [0]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode

import folium
from folium import *
from folium.plugins import *

import os
from pathlib import Path

ประกาศฟังก์ชันในการแสดงแผนที่โดยใช้ HTML iframe แต่ Colab ไม่ Support iframe เราจะ return Map ออกไปเลย แต่มีข้อเสียคือถ้า Map ข้อมูลมากเกินไปอาจจะไม่โชว์เลย

In [0]:
from IPython.display import IFrame, HTML

def embed_map(m, file_name):    
    m.save(file_name)
    # # VM
    # return IFrame(src=file_name, width='100%', height='500px')

    # Colab
    return m

# 2. เตรียม Path สำหรับดาวน์โหลดข้อมูล

กำหนด path ของ Config File และ Dataset ว่าจะอยู่ใน Google Drive ถ้าเราใช้ Google Colab หรือ อยู่ใน HOME ถ้าเราใช้ VM ธรรมดา และกำหนด Environment Variable ไปยังโฟลเดอร์ที่เก็บ kaggle.json

ในกรณีใช้ Colab ให้ Mount Google Drive เพื่อดึง Config File มาจาก Google Drive ส่วนตัวของเรา เมื่อเรารัน Cell ด้านล่างจะมีลิงค์ปรากฎขึ้นมาให้เรา Login กด Approve แล้ว Copy Authorization Code มาใส่ในช่องด้านล่าง แล้วกด Enter

In [8]:
dataset = 'alexisbcook/geospatial-learn-course-data'

# Google Colab
config_path = Path('/content/drive')
data_path = Path('/content/datasets/')/dataset
from google.colab import drive
drive.mount(str(config_path))
os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/My Drive/.kaggle"

## VM
# config_path = Path(os.getenv("HOME"))
# data_path = config_path/"datasets"/dataset
# data_path.mkdir(parents=True, exist_ok=True)
# os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/.kaggle"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3. Geocoding

เรียกฟังก์ชัน Geocode เพื่อแปลง ข้อความ เป็น พิกัด และ ที่อยู่ โดยที่เราต้องใส่ Parameter

* string ข้อความระบุ ชื่อสถานที่ หรือ ที่อยู่ 
* provider จะไปดึงข้อมูลจากไหน ในเคสนี้เราจะใช้ nominatim จาก OpenStreetMap จะได้ไม่ต้องสมัครสมาชิก เพื่อขอ API Key

ถ้า Geocode ได้สำเร็จ ระบบจะตอบรับ return ข้อมูลดังนี้

* geometry คือ Point ระบุ Latitude, Longitude
* address คือ ที่อยู่เต็ม 



In [9]:
result = geocode("Eiffel Tower", provider="nominatim", user_agent='BUA Labs/0.0.1')
result

,geometry,address
0,POINT (2.29450 48.85826),"Tour Eiffel, 5, Avenue Anatole France, Quartie..."


ผลลัพธ์ คือ พิกัด และที่อยู่ ของ หอไอเฟล แห่งกรุงปารีส ประเทศฝรั่งเศส

In [10]:
point = result.iloc[0].geometry
print(f'Latitude: {point.y}')
print(f'Longitude: {point.x}')
print(f'Address: {result.iloc[0].address}')

Latitude: 48.8582602
Longitude: 2.29449905431968
Address: Tour Eiffel, 5, Avenue Anatole France, Quartier du Gros-Caillou, Paris 7e Arrondissement, Paris, Île-de-France, France métropolitaine, 75007, France


# 4. Dataset

ในเคสนี้ เราจะใช้ Dataset มหาวิทยาลัยชั้นนำจากทั่วโลก

Dataset เราจะดึงจาก Kaggle วิธี Download kaggle.json ให้ดูจาก ep ที่แล้ว

เมื่อได้ kaggle.json มาแล้ว ในกรณีใช้ Google Colab ให้นำมาใส่ไว้ในโฟลเดอร์ My Drive/.kaggle ใน Google Drive ของเรา เป็น My Drive/.kaggle/kaggle.json ถ้าใช้ VM ให้ใส่ใน HOME/.kaggle/

สั่งดาวน์โหลด Dataset จาก Kaggle พร้อมทั้ง unzip ไว้ใน data_path

In [11]:
!kaggle datasets download {dataset} -p "{data_path}" --unzip

 96% 225M/233M [00:06<00:00, 25.8MB/s]
100% 233M/233M [00:06<00:00, 37.7MB/s]


โหลดข้อมูล มหาวิทยาลัยชั้นนำจากทั่วโลก

In [20]:
universities = pd.read_csv(data_path/"top_universities.csv")
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


จำนวน 100 มหาวิทยาลัย

In [21]:
len(universities)

100

ประกาศฟังก์ชัน สำหรับเรียก geocode แล้วแปลงเป็น Pandas Series Latitude, Longitude

In [0]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim', user_agent='BUA Labs/0.0.1').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

เราจะใช้ชื่อมหาวิทยาลัย ไปเรียก Geocode เพื่อนำพิกัดมาใส่ใน universities Pandas Dataframe

In [23]:
universities[['Latitude', 'Longitude', 'geometry']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)
universities.head()

,Name,Latitude,Longitude,geometry
0,University of Oxford,51.753454,-1.254010,POINT (-1.25400997048855 51.7534538)
1,University of Cambridge,52.176390,0.143089,POINT (0.143088815415187 52.17638955)
2,Imperial College London,51.498871,-0.175608,POINT (-0.175607955839404 51.49887085)
3,ETH Zurich,47.376453,8.547709,POINT (8.54770931489751 47.3764534)
4,UCL,51.521682,-0.135208,POINT (-0.1352078 51.5216821)


หาว่ามหาวิทยาลัยไหนบ้างที่ Geocode ไม่ขึ้น ไม่พบพิกัด เป็นกี่เปอร์เซ็นของทั้งหมด

In [24]:
print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

89.0% of addresses were geocoded!


ตอนนี้เราจะลบ มหาวิทยาลัยที่ Geocode ไม่เจอทิ้งไปก่อน

In [25]:
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(universities, geometry=universities.geometry)
universities.crs = {'init': 'epsg:4326'}
universities.head()

,Name,Latitude,Longitude,geometry
0,University of Oxford,51.753454,-1.254010,POINT (-1.25401 51.75345)
1,University of Cambridge,52.176390,0.143089,POINT (0.14309 52.17639)
2,Imperial College London,51.498871,-0.175608,POINT (-0.17561 51.49887)
3,ETH Zurich,47.376453,8.547709,POINT (8.54771 47.37645)
4,UCL,51.521682,-0.135208,POINT (-0.13521 51.52168)


In [26]:
len(universities)

89

เหลือ 89 มหาวิทยาลัย

# 5. Plot Map

นำข้อมูลมหาวิทยาลัยด้านบน มาพล็อตบนแผนที่

In [0]:
# Create a map
m_1 = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m_1)

# Display the map
embed_map(m_1, '25e-m_1.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/10/25e-m_1.html" frameborder="0" allowfullscreen></iframe>

# 6. Table Join

In [0]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world.loc[world.continent == 'Europe'].reset_index(drop=True)

europe_stats = europe[["name", "pop_est", "gdp_md_est"]]
europe_boundaries = europe[["name", "geometry"]]

In [30]:
europe_boundaries.head()

,name,geometry
0,Russia,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516..."
1,Norway,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ..."
2,France,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ..."
3,Sweden,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300..."
4,Belarus,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372..."


In [31]:
europe_stats.head()

,name,pop_est,gdp_md_est
0,Russia,142257519,3745000.0
1,Norway,5320045,364700.0
2,France,67106161,2699000.0
3,Sweden,9960487,498100.0
4,Belarus,9549747,165400.0


In [47]:
europe = europe_boundaries.merge(europe_stats, on="name")
europe.set_index('name', inplace=True)
europe.head()

,geometry,pop_est,gdp_md_est
name,,,
Russia,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516...",142257519,3745000.0
Norway,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ...",5320045,364700.0
France,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ...",67106161,2699000.0
Sweden,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300...",9960487,498100.0
Belarus,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372...",9549747,165400.0


นำมาข้อมูลจำนวนประชากรยุโรป มาพล็อตแผนที่รวมกับพิกัดมหาวิทยาลัยชั้นนำ จะเห็นว่ามีหลุดไปทวีปอื่น ๆ บ้าง

In [0]:
# Create a base map
m_2 = folium.Map(location=[54, 15], tiles='cartodbpositron', zoom_start=3)

# create a map
Choropleth(geo_data=europe.geometry.__geo_interface__, data=europe.pop_est, key_on='feature.id', 
           fill_color='YlGnBu', 
           legend_name='Population').add_to(m_2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m_2)

embed_map(m_2, '25e-m_2.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/10/25e-m_2.html" frameborder="0" allowfullscreen></iframe>

# 7. Spatial Join

เราจะใช้ Spatial Join ในการ Match พิกันของมหาวิทยาลัยเข้า กับขอบเขตของประเทศในทวีปยุโรป

In [53]:
european_universities = gpd.sjoin(universities, europe)
european_universities.head()

,Name,Latitude,Longitude,geometry,index_right,pop_est,gdp_md_est
0,University of Oxford,51.753454,-1.254010,POINT (-1.25401 51.75345),United Kingdom,64769452,2788000.0
1,University of Cambridge,52.176390,0.143089,POINT (0.14309 52.17639),United Kingdom,64769452,2788000.0
2,Imperial College London,51.498871,-0.175608,POINT (-0.17561 51.49887),United Kingdom,64769452,2788000.0
4,UCL,51.521682,-0.135208,POINT (-0.13521 51.52168),United Kingdom,64769452,2788000.0
5,London School of Economics and Political Science,51.514429,-0.116588,POINT (-0.11659 51.51443),United Kingdom,64769452,2788000.0


จาก 89 มหาวิทยาลัยในตอนแรก เหลือ 83 มหาวิทยาลัย ใน 15 ประเทศ ในทวีปยุโรป

In [35]:
print("We located {} universities.".format(len(universities)))
print("Only {} of the universities were located in Europe (in {} different countries).".format(
    len(european_universities), len(european_universities.name.unique())))


We located 89 universities.
Only 83 of the universities were located in Europe (in 15 different countries).


นำมาพล็อตลงแผนที่ ว่าจะเหลือแต่ในทวีปยุโรปไหม

In [0]:
# Create a base map
m_3 = folium.Map(location=[54, 15], tiles='cartodbpositron', zoom_start=3)

# create a map
Choropleth(geo_data=europe.geometry.__geo_interface__, data=europe.pop_est, key_on='feature.id', 
           fill_color='YlGnBu', 
           legend_name='Population').add_to(m_3)

# Add points to the map
for idx, row in european_universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m_3)

embed_map(m_3, '25e-m_3.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/10/25e-m_3.html" frameborder="0" allowfullscreen></iframe>

# Credit

* https://www.kaggle.com/alexisbcook/manipulating-geospatial-data
* http://geopandas.org/geocoding.html
* http://geopandas.org/mergingdata.html
* https://www.bualabs.com/archives/2744/geospatial-analysis-crimes-in-boston-usa-plot-interactive-map-crime-prevention-law-enforcement-folium-geospatial-ep-4/